In [1]:
import json
import sys
import os
import numpy as np
from scipy import ndimage
from scipy import special
from scipy import sparse
import anndata
import pandas as pd
import time
from sklearn.decomposition import PCA
import collections
import pickle as pkl
import zarr
import h5py
import anndata as ad

from dredFISH.Analysis.__init__plots import *
sns.set_style('white')

import importlib
from dredFISH.Analysis import basicu
importlib.reload(basicu)

<module 'dredFISH.Analysis.basicu' from '/bigstore/GeneralStorage/fangming/projects/dredfish/packages/dredFISH/dredFISH/Analysis/basicu.py'>

In [2]:
ddir = '/bigstore/binfo/mouse/Brain/Sequencing/Allen_10X_CtxHippo/source/'
outdir = '/scratch/10x/'

# preload data

In [3]:
ti = time.time()
f = ddir + 'expression_matrix.hdf5'
with h5py.File(f, 'r') as fh:
    print(fh.keys())
    print(fh['data'].keys())
    
    print(fh['data']['counts'])
    print(fh['data']['gene'])# t is just the transpose
    print(fh['data']['samples'])
    print(fh['data']['shape'])
    
    print(fh['data']['shape'][()])
    
    # mat = fh['data']['counts'][:,:100] # [gene, cell]
    
    genes = fh['data']['gene'][()]
    genes = np.array([gene.decode() for gene in genes])
    
    cells = fh['data']['samples'][()]
    cells = np.array([cell.decode() for cell in cells])
    
genes_10x = genes.copy()

<KeysViewHDF5 ['data']>
<KeysViewHDF5 ['counts', 'gene', 'samples', 'shape']>
<HDF5 dataset "counts": shape (31053, 1169320), type "<i4">
<HDF5 dataset "gene": shape (31053,), type "|S30">
<HDF5 dataset "samples": shape (1169320,), type "|S36">
<HDF5 dataset "shape": shape (2,), type "<i4">
[1169320   31053]


# load, save

In [4]:
# %%time

# ncells = len(cells)
# # chunksize = 100
# chunksize = 100000
# curr_head = 0
# print(ncells)
# f = ddir + 'expression_matrix.hdf5'


# with h5py.File(f, 'r') as fh:
#     while curr_head < ncells:
#         l, r = curr_head, curr_head+chunksize
#         r = min(ncells, r)
#         curr_head += chunksize
#         print(l,r)
        
#         # get matrix
#         mat = fh['data']['counts'][:,l:r] # [gene, cell]
        
#         # process
#         Xcell = mat.T
#         print(Xcell.shape)

# #         # # correct for library size (normalization) and projection 
# #         libsizes = Xcell.sum(axis=1)
# #         medsize = np.median(libsizes)

# #         Xcr_norm = (Xcell[:,genes_idx]/libsizes.reshape(-1,1))*medsize
# #         Xcr_norm = Xcr_norm.dot(pmat)

#         # save to scratch
#         # np.save(f'{outdir}{l}.npy', Xcell)
        
#         break


In [5]:
%%time
with h5py.File(f, 'r') as fh:
    # get matrix
    mat = fh['data']['counts'][:,:100] # [gene, cell]

    # process
    Xcell = mat.T
    print(Xcell.shape)


(100, 31053)
CPU times: user 2.64 s, sys: 564 ms, total: 3.21 s
Wall time: 3.2 s


In [6]:
dat_dir = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/'
output = os.path.join(dat_dir, 'rna', 'scrna_10x_ctxhippo_a_exon_count_matrix_v2.zarr')
print(output)

/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_10x_ctxhippo_a_exon_count_matrix_v2.zarr


In [7]:
%%time
chunksize = 100
ncells = len(cells)
ngenes = len(genes)
print(f'{ncells:,}, {ngenes:,}')

# create a zarr file (dir)
z1 = zarr.open(output, mode='w')
z1

1,169,320, 31,053
CPU times: user 232 µs, sys: 7.26 ms, total: 7.49 ms
Wall time: 40.3 ms


<zarr.hierarchy.Group '/'>

In [8]:
%%time
# create count matrix
z1.create_dataset('counts', 
                  shape=(len(Xcell), ngenes), 
                  chunks=(chunksize, None), 
                  dtype='i4',
                 )
z1['counts'][:] = Xcell

CPU times: user 35.9 ms, sys: 7.19 ms, total: 43.1 ms
Wall time: 64.7 ms
